*SU(4) to SU(2)_S x SU(2)_T decomposition*  

In [5]:
#########################################################################################################
## Calculate  SU(4) to SU(2)_S x SU(2)_T decomposition                                                 ##
##                                                                                                     ##
## Args:                                                                                               ## 
##        SU$_irrep: Valid U(6) irrep (4 non-increasing non-negative integers:f1,f2,f3,f4)             ##
##                                                                                                     ##
##    Returns:                                                                                         ##
##        DataFrame with quantum numbers and dimensions                                                ##
##                                                                                                     ##
##    Developed by J.P. Valencia Valencia, S. Cordoba and R. Henao                                     ## 
#########################################################################################################  
import os, sys
module_path = os.path.abspath("/home/pvalen/su4-branching/su4_branching") #<-----use the path to modules
if module_path not in sys.path:
    sys.path.append(module_path)
import su4_branching 
#from su4_branching import racah_su4_to_st, u6_to_su4_irrep
print("✓ Module imported successfully!")
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)
 
from su4_export import export_su4_with_labels, export_multiple_representations
from pathlib import Path
from typing import Tuple
from fractions import Fraction
# ----- ---------------------------------------------------------#<------ COMPUTE THE DATA -------------
f1, f2, f3, f4 = 8,5,5,0                                    # <-- choose your representation here
su4_df, st_df   = su4_branching.racah_su4_to_st(f1, f2, f3, f4)  # <--show the dataframe

###-------------------------------export-representation to fomat latex-cvs------------------
### Single representation
files = export_su4_with_labels(f1, f2, f3, f4, su4_branching)    #<-only one representation 

### Multiple representations
reps = [(5,1,0,0),(4,2,0,0),(4,1,1,0),(5,3,2,0), (8,4,2,0), (9,4,2,0)]                         
results = export_multiple_representations(reps, su4_branching)  #<---multiple representations

✓ Module imported successfully!
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,8,5,5,0,],(,4,4,-1,),(,3,0,5,),228.0,770



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,1,1,3,3
1,0,3,1,7,10
2,1,0,1,3,13
3,1,1,1,9,22
4,1,2,2,30,52
5,1,3,1,21,73
6,1,4,1,27,100
7,2,1,2,30,130
8,2,2,2,50,180
9,2,3,2,70,250


Processing SU(4) representation [8,5,5,0]...
Exporting CSV files...
Exporting LaTeX files...

Files created for representation [8,5,5,0]:
 • su4_representation_8_5_5_0.csv
 • su4_branching_rules_8_5_5_0.csv
 • su4_representation_8_5_5_0.tex
 • su4_branching_rules_8_5_5_0.tex

LaTeX labels:
 • tab:su4_info_8_5_5_0
 • tab:branching_rules_8_5_5_0
Batch export: 6 representations
--------------------------------------------------
[ 1/6] [5,1,0,0]... ✓
[ 2/6] [4,2,0,0]... ✓
[ 3/6] [4,1,1,0]... ✓
[ 4/6] [5,3,2,0]... ✓
[ 5/6] [8,4,2,0]... ✓
[ 6/6] [9,4,2,0]... ✓
--------------------------------------------------
Summary: 6 successful, 0 failed


*SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(6) irrep (ds-shell).*

In [6]:
###########################################################################
##Calculate SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(6) irrep     ##
##Developed by J.P. Valencia, S. Cordoba, and R. Henao                   ##
##                                                                       ##
##Convert U(6) Young tableau to SU(4) irrep via conjugate partition.     ##   
##    Args:                                                              ##
##        u6_irrep: U(6) irrep as 6 non-increasing non-negative integers ##       
##    Returns:                                                           ##
##        (f1, f2, f3, f4) - SU(4) irrep labels                          ##
##    Raises:                                                            ## 
##        SymmetryError: If input violates U(6) constraints              ##
###########################################################################     
class SymmetryError(ValueError):
    """Exception for invalid symmetry inputs."""
    pass


def u6_to_su4_irrep(u6_irrep: Tuple[int, ...]) -> Tuple[int, int, int, int]:
    
    # Validate input format and constraints
    if len(u6_irrep) != 6 or any(not isinstance(x, int) or x < 0 for x in u6_irrep):
        raise SymmetryError("Input must be 6 non-negative integers")
    
    if list(u6_irrep) != sorted(u6_irrep, reverse=True):
        raise SymmetryError("Young diagram must be non-increasing sequence")
    
    # Compute conjugate partition (transpose Young diagram)
    current = np.array(u6_irrep, dtype=np.int32)
    su4_irrep = []
    
    while np.any(current > 0):
        # Count non-zero elements in current column
        non_zero_count = np.count_nonzero(current)
        su4_irrep.append(non_zero_count)
        # Remove first column and decrement remaining elements
        current = current[current > 0] - 1
    
    # Enforce SU(4) constraint: first column has at most 4 boxes
    # Remove boxes beyond f4 position
    while len(su4_irrep) >= 4:
        last_column_height = su4_irrep[-1]
        su4_irrep = [x - last_column_height for x in su4_irrep[:-1]]
    
    # Pad with zeros to ensure 4 elements
    su4_irrep.extend([0] * (4 - len(su4_irrep)))
    
    f1, f2, f3, f4 = tuple(int(x) for x in su4_irrep[:4])
    
    return f1, f2, f3, f4

#################################################################################
##    Calculate SU(4) → SU(2)_S ⊗ SU(2)_T branching from U(6) irrep.           ##
##    This function automates the conversion from orbital U(6) classification  ## 
##    to spin-isospin SU(4) decomposition, then computes all (S,T) multiplets  ##
##    and their multiplicities.                                                ##
##    Args:                                                                    ##
##        U6_irrep: U(6) irrep specified as 6 non-negative integers in         ##
##                 non-increasing order (Young tableau format)                 ## 
##    Returns:                                                                 ##
##        pd.DataFrame with columns:                                           ## 
##            - (S, T): Spin-isospin quantum numbers                           ##
##            - Multiplicity: Inner multiplicity N^{S,T}_{f1,f2,f3,0}          ##
##            - Dimension: (2S+1)(2T+1)                                        ## 
##    Raises:                                                                  ##
##        SymmetryError: If U6_irrep violates Young tableau constraints        ##
##        ValueError: If SU(4) irrep produces no valid branching               ##
##   Examples:                                                                 ##
##        >>> U6_irrep = (4, 2, 0, 0, 0, 0)  # d-shell configuration           ##
##        >>> result_df = U6_RacahSU4toST(U6_irrep)                            ## 
##        >>> print(result_df)                                                 ##        
##    Notes:                                                                   ##
##        - Uses conjugate partition to convert U(6) Young diagram to SU(4)    ##
##        - Enforces SU(4) constraint: first column ≤ 4 boxes (2S+1 ≤ 4)       ##
##        - All results validated via dimensional consistency check            ## 
##      Developed by J.P. Valencia, S. Cordoba, and R. Henao                   ##
#################################################################################

def U6_RacahSU4toST(U6_irrep: Tuple[int, int, int, int, int, int]) -> pd.DataFrame:        
    # Step 1: Convert U(6) irrep to SU(4) labels
    f1, f2, f3, f4 = u6_to_su4_irrep(U6_irrep)
    
    # Step 2: Validate SU(4) irrep (redundant but explicit)
    if not (f1 >= f2 >= f3 >= f4 >= 0):
        raise SymmetryError(f"Invalid SU(4) irrep: ({f1}, {f2}, {f3}, {f4})")
    
    # Step 3: Call branching calculation
    try:
        branching_results = su4_branching.racah_su4_to_st(f1, f2, f3, f4)
    except Exception as e:
        raise ValueError(
            f"Failed to compute SU(4)→(S,T) branching for [{f1},{f2},{f3},{f4}]: {e}"
        )
    
    # Step 4: Format output DataFrame
    if isinstance(branching_results, tuple) and len(branching_results) == 2:
        su4_df, st_df = branching_results
        # Use the ST dataframe (contains spin-isospin decomposition)
        output_df = st_df.copy()
    else:
        output_df = branching_results
    
    # Add metadata columns for traceability
    output_df.attrs['U6_irrep'] = tuple(U6_irrep)
    output_df.attrs['SU4_irrep'] = (f1, f2, f3, f4)
    output_df.attrs['N_total'] = sum(U6_irrep)  # Total number of particles
    
    return output_df


# ============================================================================
# EXAMPLE USAGE
# ============================================================================

if __name__ == "__main__":
    import sys
    
    print("=" * 70)
    print("SU(4) Branching Calculator: U(6) → SU(4) → SU(2)_S ⊗ SU(2)_T")
    print("=" * 70)
    
    # Example 1: sd-shell with 4 nucleons
    u6_ex1 = (4, 0, 0, 0, 0, 0)
    print(f"\nExample 1: U(6) irrep {u6_ex1}")
    print("-" * 70)
    try:
        df1 = U6_RacahSU4toST(u6_ex1)

    except Exception as e:
        print(f"Error: {e}")
    
    # Example 2: More complex configuration
    u6_ex2 = (3, 2, 1, 0, 0, 0)
    print(f"\n\nExample 2: U(6) irrep {u6_ex2}")
    print("-" * 70)
    try:
        df2 = U6_RacahSU4toST(u6_ex2)
    except Exception as e:
        print(f"Error: {e}")

    # Example 3: Elimination-first column with length=4----
    u6_ex3 = (4, 4, 3, 1, 0, 0)
    print(f"\n\nExample 2: U(6) irrep {u6_ex3}")
    print("-" * 70)
    try:
        df3 = U6_RacahSU4toST(u6_ex3)
    except Exception as e:
        print(f"Error: {e}")    
    
    print("\n" + "=" * 70)

SU(4) Branching Calculator: U(6) → SU(4) → SU(2)_S ⊗ SU(2)_T

Example 1: U(6) irrep (4, 0, 0, 0, 0, 0)
----------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,0,0,0,0,],(,0,0,0,),(,0,0,0,),0.0,1



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,0,1,1,1




Example 2: U(6) irrep (3, 2, 1, 0, 0, 0)
----------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


...

,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,1,1,3,3
1,0,2,1,5,8
2,1,0,1,3,11
3,1,1,2,18,29
4,1,2,1,15,44
5,2,0,1,5,49
6,2,1,1,15,64




Example 2: U(6) irrep (4, 4, 3, 1, 0, 0)
----------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,2,1,1,0,],(,1,1,0,),(,1,0,1,),32.0,15



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,1,1,3,3
1,1,0,1,3,6
2,1,1,1,9,15


*SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(10) irrep (pf-shell)*

In [7]:
###########################################################################################
##Calculate SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(10) irrep (pf-shell)         ##
##Developed by J.P. Valencia, S. Cordoba, and R. Henao                                   ##
##For use in pf-shell nuclear structure calculations                                     ##
##    Convert U(10) Young tableau to SU(4) irrep via conjugate partition.                ##
##    Used for pf-shell nuclei (0f₇/₂, 1p₃/₂, 1p₁/₂, 0f₅/₂).                             ##
##    The U(10) group classifies spatial orbital configurations, while SU(4)             ##
##    describes the spin-isospin quantum numbers (S, T).                                 ##
##    Args:                                                                              ##
##        u10_irrep: U(10) irrep as 10 non-increasing non-negative integers              ##
##                   representing Young tableau row lengths                              ##
##    Returns:                                                                           ## 
##        (f1, f2, f3, f4) - SU(4) irrep labels satisfying f1 ≥ f2 ≥ f3 ≥ f4 ≥ 0         ##
##    Raises:                                                                            ## 
##        SymmetryError: If input violates U(10) Young tableau constraints               ##
##    Notes:                                                                             ## 
##        - U(10) has 10 single-particle orbital states: 0f₇/₂ (8-fold), 1p₃/₂ (2-fold)  ##
##        - For pf-shell: typically consider U(8) or U(10) with practical constraints    ##
##        - Conversion uses conjugate partition (transpose of Young diagram)             ##
##        - SU(4) constraint: f1 ≤ 4 (limited by 2S+1 ≤ 4 for fermionic systems)         ## 
##        - U(10) constraint: f1 ≤ 10 (limited by number of single-particle states)      ##
##    Example:                                                                           ##
##        >>> u10_ex = (3, 2, 1, 0, 0, 0, 0, 0, 0, 0)  # 6 nucleons                      ##
##        >>> f1, f2, f3, f4 = u10_to_su4_irrep(u10_ex)                                  ##
##        >>> print(f"SU(4) irrep: [{f1}, {f2}, {f3}, {f4}]")                            ##
###########################################################################################    

class SymmetryError(ValueError):
    """Exception for invalid symmetry inputs."""
    pass


def u10_to_su4_irrep(u10_irrep: Tuple[int, ...]) -> Tuple[int, int, int, int]:   
    # Validate input format and constraints
    if len(u10_irrep) != 10 or any(not isinstance(x, int) or x < 0 for x in u10_irrep):
        raise SymmetryError("Input must be 10 non-negative integers")
    
    if list(u10_irrep) != sorted(u10_irrep, reverse=True):
        raise SymmetryError("Young diagram must be non-increasing sequence")
    
    if max(u10_irrep) > 10:
        raise SymmetryError("U(10) Young diagram row length cannot exceed 10")
    
    # Compute conjugate partition (transpose Young diagram)
    # This converts U(N) representation to its dual SU(4) representation
    current = np.array(u10_irrep, dtype=np.int32)
    su4_irrep = []
    
    while np.any(current > 0):
        # Count non-zero elements in current column
        non_zero_count = np.count_nonzero(current)
        su4_irrep.append(non_zero_count)
        # Remove first column and decrement remaining elements
        current = current[current > 0] - 1
    
    # Enforce SU(4) constraint: first column has at most 4 boxes
    # This reflects the fact that fermionic SU(4) has dimension 4
    while len(su4_irrep) >= 4:
        last_column_height = su4_irrep[-1]
        su4_irrep = [x - last_column_height for x in su4_irrep[:-1]]
    
    # Pad with zeros to ensure 4 elements
    su4_irrep.extend([0] * (4 - len(su4_irrep)))
    
    f1, f2, f3, f4 = tuple(int(x) for x in su4_irrep[:4])
    
    # Additional validation: check SU(4) constraint
    if f1 > 4:
        raise SymmetryError(
            f"SU(4) irrep violates constraint: f1={f1} > 4. "
            f"This indicates invalid U(10) → SU(4) mapping."
        )
    
    return f1, f2, f3, f4

########################################################################################
##    Calculate SU(4) → SU(2)_S ⊗ SU(2)_T branching from U(10) irrep.                 ##
##    This function automates the conversion from orbital U(10) classification        ##
##    (used for pf-shell nuclei) to spin-isospin SU(4) decomposition, then computes   ##
##    all (S,T) multiplets and their multiplicities.                                  ##
##    The pf-shell contains four single-particle orbitals:                            ##
##    - 0f₇/₂ (8-fold degenerate, j=7/2)                                              ##
##    - 1p₃/₂ (2-fold degenerate, j=3/2)                                              ##
##    - 1p₁/₂ (0-fold... usually omitted or handled separately)                       ##
##    - 0f₅/₂ (6-fold degenerate, j=5/2)                                              ##
##    Total: 10 single-particle states (hence U(10)                                   ##
##    Args:                                                                           ##
##        U10_irrep: U(10) irrep specified as 10 non-negative integers in             ##
##                  non-increasing order (Young tableau format).                      ##
##                  Example: (5, 3, 2, 0, 0, 0, 0, 0, 0, 0) for 10 nucleons           ##  
##    Returns:                                                                        ## 
##        pd.DataFrame with columns:                                                  ##
##            - (S, T): Spin-isospin quantum numbers                                  ##
##            - Multiplicity: Inner multiplicity N^{S,T}_{f1,f2,f3,0}                 ##
##            - Dimension: (2S+1)(2T+1)                                               ##
##            - Nucleon_Count: Total number of valence nucleons                       ##
##    Raises:                                                                         ##
##        SymmetryError: If U10_irrep violates Young tableau constraints              ## 
##        ValueError: If SU(4) irrep produces no valid branching                      ##
##    Examples:                                                                       ##
##        >>> # Example 1: 6 nucleons in pf-shell                                     ##
##        >>> U10_1 = (3, 2, 1, 0, 0, 0, 0, 0, 0, 0)                                  ##
##        >>> df1 = U10_RacahSU4toST(U10_1)                                           ##
##        >>> print(f"SU(4) irrep: {df1.attrs['SU4_irrep']}")                         ##
##        >>> # Example 2: 10 nucleons in pf-shell (maximum)                          ##
##        >>> U10_2 = (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)                                  ##
##        >>> df2 = U10_RacahSU4toST(U10_2)                                           ##
##    Notes:                                                                          ##
##        - Uses conjugate partition to convert U(10) Young diagram to SU(4)          ##
##        - Enforces SU(4) constraint: first column ≤ 4 boxes (2S+1 ≤ 4)              ##
##        - All results validated via dimensional consistency check                   ##
##        - Typical pf-shell nuclei have 1-10 valence nucleons                        ##
##Developed by J.P. Valencia, S. Cordoba, and R. Henao                                ##
########################################################################################    
def U10_RacahSU4toST(U10_irrep: Tuple[int, int, int, int, int, int, int, int, int, int]) -> pd.DataFrame:    
    # Step 1: Convert U(10) irrep to SU(4) labels
    f1, f2, f3, f4 = u10_to_su4_irrep(U10_irrep)
    
    # Step 2: Validate SU(4) irrep (redundant but explicit)
    if not (f1 >= f2 >= f3 >= f4 >= 0):
        raise SymmetryError(f"Invalid SU(4) irrep: ({f1}, {f2}, {f3}, {f4})")
    
    # Step 3: Call branching calculation
    try:
        branching_results = su4_branching.racah_su4_to_st(f1, f2, f3, f4)
    except Exception as e:
        raise ValueError(
            f"Failed to compute SU(4)→(S,T) branching for [{f1},{f2},{f3},{f4}]: {e}"
        )
    
    # Step 4: Format output DataFrame
    if isinstance(branching_results, tuple) and len(branching_results) == 2:
        su4_df, st_df = branching_results
        # Use the ST dataframe (contains spin-isospin decomposition)
        output_df = st_df.copy()
    else:
        output_df = branching_results
    
    # Step 5: Add metadata for traceability and physics context
    n_particles = sum(U10_irrep)
    output_df.attrs['U10_irrep'] = tuple(U10_irrep)
    output_df.attrs['SU4_irrep'] = (f1, f2, f3, f4)
    output_df.attrs['N_valence'] = n_particles
    output_df.attrs['shell'] = 'pf-shell'
    output_df.attrs['description'] = f'U(10)⊗SU(4) branching for pf-shell with {n_particles} nucleons'
    
    return output_df


# ============================================================================
# EXAMPLE USAGE AND BENCHMARKS
# ============================================================================

if __name__ == "__main__":
    
    print("=" * 80)
    print("SU(4) Branching Calculator (pf-shell): U(10) → SU(4) → SU(2)_S ⊗ SU(2)_T")
    print("=" * 80)
    print("\nNote: pf-shell has 10 single-particle orbitals:")
    print("  - 0f₇/₂ (8 states), 1p₃/₂ (2 states)")
    print("\nTypical configurations have N_valence = 1-10 nucleons")
    print("=" * 80)
    
    # Example 1: Light pf-shell nuclei (4 nucleons)
    u10_ex1 = (2, 1, 1, 0, 0, 0, 0, 0, 0, 0)
    print(f"\nExample 1: U(10) irrep {u10_ex1} (4 nucleons)")
    print("-" * 80)
    try:
        df1 = U10_RacahSU4toST(u10_ex1)
        
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 2: Medium pf-shell nuclei (6 nucleons)
    u10_ex2 = (2, 2, 1, 1, 0, 0, 0, 0, 0, 0)
    print(f"\n\nExample 2: U(10) irrep {u10_ex2} (6 nucleons)")
    print("-" * 80)
    try:
        df2 = U10_RacahSU4toST(u10_ex2)
    
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 3: Heavy pf-shell nuclei (10 nucleons - maximum)
    u10_ex3 = (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
    print(f"\n\nExample 3: U(10) irrep {u10_ex3} (10 nucleons, fully occupied)")
    print("-" * 80)
    try:
        df3 = U10_RacahSU4toST(u10_ex3)
        
    except Exception as e:
        print(f"Error: {e}")

    # Example 4: High-dimensional SU(4) irrep (for computational testing)
    u10_ex4 = (3, 3, 2, 2, 0, 0, 0, 0, 0, 0)
    print(f"\n\nExample 4: U(10) irrep {u10_ex4} (10 nucleons, high-dim SU(4))")
    print("-" * 80)
    try:
        df4 = U10_RacahSU4toST(u10_ex4)
        
        if len(df4) > 10:
            print(f"... ({len(df4) - 10} more rows)")
    except Exception as e:
        print(f"Error: {e}")
        
    # Example 5: High-dimensional SU(4) irrep (elimination fir colum len four of SU(4))
    u10_ex5 = (4, 4, 3, 2, 0, 0, 0, 0, 0, 0)
    print(f"\n\nExample 5: U(10) irrep {u10_ex5} (10 nucleons, high-dim SU(4))")
    print("-" * 80)
    try:
        df5 = U10_RacahSU4toST(u10_ex5)
        
        if len(df5) > 10:
            print(f"... ({len(df5) - 10} more rows)")
    except Exception as e:
        print(f"Error: {e}")

    


SU(4) Branching Calculator (pf-shell): U(10) → SU(4) → SU(2)_S ⊗ SU(2)_T

Note: pf-shell has 10 single-particle orbitals:
  - 0f₇/₂ (8 states), 1p₃/₂ (2 states)

Typical configurations have N_valence = 1-10 nucleons

Example 1: U(10) irrep (2, 1, 1, 0, 0, 0, 0, 0, 0, 0) (4 nucleons)
--------------------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,3,1,0,0,],(,2,1,1,),(,2,1,0,),64.0,45



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,1,1,3,3
1,1,0,1,3,6
2,1,1,1,9,15
3,1,2,1,15,30
4,2,1,1,15,45




Example 2: U(10) irrep (2, 2, 1, 1, 0, 0, 0, 0, 0, 0) (6 nucleons)
--------------------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


...

,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,0,1,1,1
1,0,2,1,5,6
2,1,1,2,18,24
3,1,2,1,15,39
4,1,3,1,21,60
5,2,0,1,5,65
6,2,1,1,15,80
7,2,2,1,25,105
8,3,1,1,21,126




Example 5: U(10) irrep (4, 4, 3, 2, 0, 0, 0, 0, 0, 0) (10 nucleons, high-dim SU(4))
--------------------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,2,2,1,0,],(,3/2,1/2,-1/2,),(,0,1,1,),39.0,20



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,1/2,1/2,1,4,4
1,1/2,3/2,1,8,12
2,3/2,1/2,1,8,20
